## Mariana Godina Abasolo
### 113682

## Ejercicio/Tarea

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?

#### Un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation

#### ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

- En este caso resultó más rápida la busqueda en secuencial que la paralela, se puede observar en los siguientes dos ejercicios el tiempo que le toma a cada uno ajustar un modelo de regresión ridge.

Se crea una variable, `total`, con la suma del `fare_amount` y `tip_amount`. Con la variable `total` obtenemos la proporción de propina, `prop_propina`.

In [1]:
from dask import dataframe
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from dask import delayed
from dask_ml.linear_model import LinearRegression
from dask_ml.model_selection import GridSearchCV as DaskGSearch

In [2]:
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df["trip_duration"] = trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime
trips_df['total'] = trips_df['fare_amount'] + trips_df['tip_amount']
trips_df['prop_propina'] = (trips_df['tip_amount']/trips_df['total'])
df_mod =  trips_df.compute()
df_mod = df_mod[df_mod["fare_amount"]>0]
df_mod["trip_duration_seg"] = df_mod.trip_duration.map(lambda d: d.total_seconds())
label_encoder = LabelEncoder()
df_mod["car_type"] = label_encoder.fit_transform(df_mod['car_type'])
df_mod.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration,total,prop_propina,trip_duration_seg
0,0,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,26.60,0.172932,1170.0
1,0,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,9.00,0.000000,587.0
2,0,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,8.50,0.117647,627.0
3,0,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,9.50,0.105263,551.0
4,0,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,9.16,0.181223,568.0


Las variables que se usarán en el modelo son:
- `car_type`
- `fare_amount`
- `passenger_count`
- `trip_distance`
- `trip_duration_seg`
- `prop_propina`

In [3]:
df_mod = df_mod.drop(["taxi_id", "tip_amount", "tpep_dropoff_datetime", "tpep_pickup_datetime", "trip_duration", "total"], axis= 1)
df_mod.head()

,car_type,fare_amount,passenger_count,trip_distance,prop_propina,trip_duration_seg
0,0,22.0,1,6.90,0.172932,1170.0
1,0,9.0,1,1.81,0.000000,587.0
2,0,7.5,1,0.96,0.117647,627.0
3,0,8.5,1,1.90,0.105263,551.0
4,0,7.5,1,1.00,0.181223,568.0


#### Tiempo secuencial

In [4]:
X = df_mod.drop("prop_propina", axis = 1)
y = df_mod["prop_propina"]
#X_train, X_test, y_train, y_test = train_test_split(X, y)

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
classifier = Ridge()
grid = GridSearchCV(classifier, param_grid=dict(alpha=alphas), cv = 10)

In [5]:
%%time
grid.fit(X, y)

CPU times: user 450 ms, sys: 440 ms, total: 890 ms
Wall time: 491 ms


GridSearchCV(cv=10, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

#### Tiempo paralelo

In [6]:
X = df_mod.drop("prop_propina", axis = 1)
y = df_mod["prop_propina"]
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
classifier = Ridge()
grid = GridSearchCV(classifier, param_grid=dict(alpha=alphas), cv = 10)


In [7]:
%%time
delayed(grid.fit(X, y))

CPU times: user 470 ms, sys: 410 ms, total: 880 ms
Wall time: 472 ms


Delayed('GridSearchCV-ee463fb9-ceab-4955-890b-bfade933e615')

##### ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

- Comprarando los tres tiempos la busqueda seceuncial sigue siendo más rápida que la paralela y la de Dask ML. Esto se puede ser porque no es lo suficiente grande la base para que se note la diferencia.


In [8]:
from dask_ml.linear_model import LinearRegression
X = df_mod.drop("prop_propina", axis = 1).values
y = df_mod["prop_propina"].values
param_grid = {'C': [1, 10, 100, 1000, 10000]}
lr = LinearRegression()
grid_d = DaskGSearch(lr, param_grid, cv=10)


In [9]:
%%time
grid_d.fit(X,y)

CPU times: user 3.74 s, sys: 5.56 s, total: 9.3 s
Wall time: 5.25 s


GridSearchCV(cache_cv=True, cv=10, error_score='raise',
       estimator=LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False),
       iid=True, n_jobs=-1, param_grid={'C': [1, 10, 100, 1000, 10000]},
       refit=True, return_train_score='warn', scheduler=None, scoring=None)